In [2]:
#Detects enotion from text
#Created by S. Biswas
import pandas as pd

In [22]:
df_train = pd.read_csv('train.txt', sep=';', names=['Sentence', 'Emotion'], encoding='UTF8')
#data cleaning
df_train = df_train[df_train['Sentence'] != '']
df_train = df_train.dropna()
df_train = df_train.drop_duplicates()
# df_train = df_train[df_train['Sentence'] != '']


In [23]:
df_train['length'] = [len(sen) for sen in df_train['Sentence']]

In [24]:
df_train.head()

,Sentence,Emotion,length
0,i didnt feel humiliated,sadness,23
1,i can go from feeling so hopeless to so damned...,sadness,108
2,im grabbing a minute to post i feel greedy wrong,anger,48
3,i am ever feeling nostalgic about the fireplac...,love,92
4,i am feeling grouchy,anger,20


In [33]:
# import matplotlib.pyplot as plt
# import seaborn as sea
# sea.countplot(x=df_train['Emotion'])
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_train['Emotion'] = lb.fit_transform(df_train['Emotion'])

In [34]:
print(df_train['Emotion'])

0        4
1        4
2        0
3        3
4        0
        ..
15995    4
15996    4
15997    2
15998    0
15999    4
Name: Emotion, Length: 15999, dtype: int64


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df_train['Cleaned_Sentence'] = df_train['Sentence'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
df_train['Final_Cleaned_Sentence'] = df_train['Cleaned_Sentence'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))



In [44]:
df_train.head()

,Sentence,Emotion,length,Cleaned_Sentence,Final_Cleaned_Sentence
0,i didnt feel humiliated,4,23,i didnt feel humiliated,didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,4,108,i can go from feeling so hopeless to so damned...,go feeling hopeless damned hopeful around some...
2,im grabbing a minute to post i feel greedy wrong,0,48,im grabbing a minute to post i feel greedy wrong,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,3,92,i am ever feeling nostalgic about the fireplac...,ever feeling nostalgic fireplace know still pr...
4,i am feeling grouchy,0,20,i am feeling grouchy,feeling grouchy


In [59]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['Final_Cleaned_Sentence'], df_train['Emotion'], test_size= 0.2 , random_state=42 )


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer()
x_train_tfidf = tfidfv.fit_transform(x_train)
x_test_tfidf = tfidfv.transform(x_test)
print(x_train_tfidf)

  (0, 12824)	0.3024031259067421
  (0, 10002)	0.3024031259067421
  (0, 11635)	0.2014964071061957
  (0, 9401)	0.17082270472767283
  (0, 7245)	0.3024031259067421
  (0, 6440)	0.3024031259067421
  (0, 10888)	0.22179505516685435
  (0, 6971)	0.3024031259067421
  (0, 9805)	0.280935619704907
  (0, 6180)	0.3024031259067421
  (0, 10877)	0.1986530295548731
  (0, 4362)	0.06734548295806918
  (0, 5435)	0.15753625405700036
  (0, 1667)	0.15531316086672445
  (0, 11881)	0.1562661918993771
  (0, 2586)	0.21121071566694555
  (0, 9537)	0.3024031259067421
  (1, 12084)	0.4183040168164712
  (1, 4364)	0.3325114392421894
  (1, 11740)	0.5502746800093348
  (1, 10880)	0.3522391416525957
  (1, 831)	0.4379464248012754
  (1, 4359)	0.09610944717451617
  (1, 13202)	0.2941923844771472
  (2, 6830)	0.3921515500410638
  :	:
  (12794, 9428)	0.45465629452962053
  (12794, 6701)	0.3470958751460806
  (12794, 2368)	0.3730084311340277
  (12794, 10704)	0.3547679894759444
  (12794, 6976)	0.2634535785669521
  (12794, 4362)	0.126453812

In [74]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
classifier = {
    'MultinomialNB' : MultinomialNB(),
    'LogisticRegression' : LogisticRegression(),   
}
for name, cls in classifier.items():
    cls.fit(x_train_tfidf, y_train)
    y_pred = cls.predict(x_test_tfidf)
    acc = cls.accuracy_score(y_test, y_pred)
    print(f"====={name}======\n")
    print(f"====={acc}======\n")
    
    

AttributeError: 'MultinomialNB' object has no attribute 'accuracy_score'